## Лекция 3  NER

In [6]:
import re
import pandas as pd
import numpy as np

In [34]:
answers = pd.read_excel('answers_base.xlsx')
queries = pd.read_excel("queries_base.xlsx")

In [35]:
answers.head()

,Номер связки,Текст вопросов,Текст ответа,Тематика
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ


In [36]:
queries.head()

,Текст вопроса,Номер связки\n,Тематика,Unnamed: 3,Unnamed: 4
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,NaN,NaN
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,NaN,NaN
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN


### __Задача 1__:

Реализуйте 2 функции препроцессинга:

- Удалить именованные сущности с помощью natasha (https://github.com/natasha/yargy)
- Удалить именованные сущности с помощью deepmipt (https://github.com/deepmipt/ner)

In [1]:
!pip install --quiet natasha

     |████████████████████████████████| 34.4MB 115kB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 8.0MB 48.0MB/s 


In [2]:
# была мысль делать с extractors, но
# показалось, что 3 задание чем-то похоже на это
# поэтому, я их не использовала

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    
    NewsNERTagger,

    PER,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    
    Doc
)

"""
инициализируем сегментатор,
эмбендинги и ner-тэггер
"""

morph_vocab = MorphVocab()


segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)
ner_tagger = NewsNERTagger(emb)

#### Natasha. Смотрим на то, что лучше сделать

In [37]:
text = queries.head(1)['Текст вопроса'].tolist()[0]
doc = Doc(text)

Тут чуть удобнее пользоваться вместо ner_tagger(text), откуда можно вытащить span вот такого типа: 

---
Span(start=12, stop=36, type='PER')


Span(start=74, stop=80, type='LOC')


Span(start=109, stop=116, type='LOC')


Span(start=206, stop=212, type='LOC')


Span(start=228, stop=242, type='PER')

---

пользоваться tag_ner(ner_tagger), но для этого нужно пришлось задействовать сегментатор, зато результат, кажется, лучше, потому что отсюда можно вытащить текст (я до конца не определилась, что лучше использовать, поэтому выбрала этот вариант):

---
DocSpan(start=12, stop=36, type='PER', text='Вероника Игоревна\xa0 Ильич', tokens=[...])


DocSpan(start=74, stop=80, type='LOC', text='Москву', tokens=[...])


DocSpan(start=109, stop=116, type='LOC', text='Молдовы', tokens=[...])


DocSpan(start=206, stop=212, type='LOC', text='Россию', tokens=[...])


DocSpan(start=228, stop=242, type='PER', text='Иван Звягинцев', tokens=[...])

---

Тут интересно, что tag_ner(ner_tagger) https://pypi.org/project/natasha/ хорошо видит именованные сущности типа организаций, людей, локаций и т.д. (но не очень хорошо видит ареса типа 2010 года (из примера с сайта)), но это не кажется проблемой, потому что в 3 задаче мы должны доработать правила

In [10]:
markup = ner_tagger(text)
markup.print()

с уважением Вероника Игоревна  Ильич
            PER─────────────────────
После 15 августа 2020 года к нам, в Москву планируют приехать гости из
                                    LOC───                            
 Молдовы.
 LOC──── 
Какой на сегодняшний день режим по Ковид распространяется на 
иностранцев, прибывающих в Россию?
                           LOC─── 
С уважением,
Иван Звягинцев
PER───────────


In [11]:
doc.segment(segmenter)

In [12]:
doc.tag_ner(ner_tagger)

In [13]:
all_spans = []
for span in doc.spans:
    all_spans.append(span.text)

In [14]:
for el in all_spans:
    text = re.sub(r'%s' % (el), '', text)

In [15]:
text

'с уважением \n\nПосле 15 августа 2020 года к нам, в  планируют приехать гости из .\nКакой на сегодняшний день режим по Ковид распространяется на иностранцев, прибывающих в ?\n\nС уважением,\n\n'

#### Функция

In [16]:
# извление именованных сущностей может быть таким
# мне показалось, что это оптимальнее, чем 
# делать extractors

def preprocess_with_natasha(text: str) -> str:
    text = text

    doc = Doc(str(text))
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    
    # список всех именованных сущностей в тексте
    all_spans = []
    for span in doc.spans:
        all_spans.append(span.text)
        
    # удаление
    clean_text = text
    
    for el in all_spans:
        clean_text = clean_text.replace(el, '')

    return clean_text

Пример на одном предложении:

In [17]:
text = queries.head(1)['Текст вопроса'].tolist()[0]
preprocess_with_natasha(text)

'с уважением \n\nПосле 15 августа 2020 года к нам, в  планируют приехать гости из .\nКакой на сегодняшний день режим по Ковид распространяется на иностранцев, прибывающих в ?\n\nС уважением,\n\n'

In [60]:
from tqdm.auto import tqdm

In [19]:
natasha_ans = []
for index, row in tqdm(answers.iterrows()):
    text = row['Текст вопросов']
    clean_nat = preprocess_with_natasha(text)
    natasha_ans.append(clean_nat)

In [20]:
answers['preprocess_natasha'] = natasha_ans

In [21]:
answers.head()

,Номер связки,Текст вопросов,Текст ответа,Тематика,preprocess_natasha
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,У ребенка в школе продлили каникулы. Могу ли я...
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ,Где сделать вакцинацию от коронавируса?\nСущес...
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ,Сколько стоит сделать вакцину от гриппа?\nМожн...
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ,Могу я отказаться от вакцинации?\nВ каких случ...
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ,Безопасна ли вакцинация?\nОпасна ли вакцинация...


In [22]:
natasha_quer = []
for index, row in tqdm(queries.iterrows()):
    text = row['Текст вопроса']
    clean_nat = preprocess_with_natasha(text)
    natasha_quer.append(clean_nat)

In [23]:
queries['preprocess_natasha'] = natasha_quer

In [24]:
queries.head()

,Текст вопроса,Номер связки\n,Тематика,Unnamed: 3,Unnamed: 4,preprocess_natasha
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN,с уважением \n\nПосле 15 августа 2020 года к н...
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,NaN,NaN,"Здравствуйте! Проинформируйте, пожалуйста, нуж..."
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,NaN,NaN,"--\nДобрый день!\n Меня, 30.07.1989г.р., п..."
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN,Добрый день.\nВ согласно постановлению необх...
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN,"Здравствуйте, в моем городе введено ограничен..."


In [211]:
# !pip install deeppavlov

In [212]:
# !python -m deeppavlov install ner_rus_bert

In [27]:
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_rus_bert, download=True)

2020-10-01 20:07:26.177 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz to /root/.deeppavlov/ner_rus_bert_v1.tar.gz
100%|██████████| 1.32G/1.32G [06:41<00:00, 3.28MB/s]
2020-10-01 20:14:08.655 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/ner_rus_bert_v1.tar.gz archive into /root/.deeppavlov/models
2020-10-01 20:14:23.944 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz to /root/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz
100%|██████████| 666M/666M [02:54<00:00, 3.81MB/s]
2020-10-01 20:17:19.245 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz archive into /root/.deeppavlov/downloads/bert_models
[nltk_data] Downloading package punkt to /ro

2020-10-01 20:17:30.660 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_rus_bert/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use stan

2020-10-01 20:17:59.446 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_rus_bert/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_rus_bert/model


In [75]:
from razdel import sentenize

#### Deeppavlov. Смотрим на то, как лучше сделать

In [78]:
text = queries.head(1)['Текст вопроса'].tolist()[0]

In [79]:
len_text = list(sentenize(text))

In [82]:
for el in len_text:
    print(el.text)

с уважением Вероника Игоревна  Ильич

После 15 августа 2020 года к нам, в Москву планируют приехать гости из Молдовы.
Какой на сегодняшний день режим по Ковид распространяется на иностранцев, прибывающих в Россию?
С уважением,
Иван Звягинцев


In [61]:
text_ind = ner_model([text])

In [63]:
new_string = []
for t in text_ind[1]:
    print()
    print(t)
    cnt = 0
    for i in t:
      if i != 'O':
          print(i, text_ind[0][0][cnt])
          new_string.append(text_ind[0][0][cnt])
      cnt += 1


['O', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O']
B-PER Вероника
I-PER Игоревна
I-PER  
I-PER Ильич
B-LOC Москву
B-LOC Молдовы
B-LOC Ковид
B-LOC Россию
B-PER Иван
I-PER Звягинцев


In [65]:
line = " ".join(l for l in text_ind[0][0] if l not in new_string)

In [66]:
text

'с уважением Вероника Игоревна\xa0 Ильич\n\nПосле 15 августа 2020 года к нам, в Москву планируют приехать гости из Молдовы.\nКакой на сегодняшний день режим по Ковид распространяется на иностранцев, прибывающих в Россию?\n\nС уважением,\nИван Звягинцев\n'

In [67]:
line

'с уважением \n \n После 15 августа 2020 года к нам , в планируют приехать гости из . \n Какой на сегодняшний день режим по распространяется на иностранцев , прибывающих в ? \n \n С уважением , \n \n'

#### Функция

In [102]:
def preprocess_with_deepmipt(text: str) -> str:
    split_text = list(sentenize(str(text)))
    fin_line = ''
    for txt in split_text:
        text_ind = ner_model([txt.text])
        new_string = []
        for t in text_ind[1]:
            cnt = 0
            for i in t:
              if i != 'O':
                  new_string.append(text_ind[0][0][cnt])
              cnt += 1
        line = " ".join(
            l for l in text_ind[0][0] if l not in new_string)
        fin_line += line + ' '
    return fin_line

На примере 1 текста:

In [91]:
# для текста в answers (12 номер)

preprocess_with_deepmipt(text)

'Как аннулировать бронирование ? Вернуть деньги за бронирование ? Как вернуть деньги за гостиницу ? Вернуть деньги за поезд ? Возврат денег за бронь ? Вернуть деньги за морскую экскурсию ? Отменили рейсы , будет ли возврат денег за билеты ? Замена билетов на поезд ? Можно ли вернуть билет на осуществление морской перевозки ? Отменили размещение в гостинице , как вернуть деньги ? Возврат ж / д билетов ? Не возвращают деньги за бронирование ? Бронировали экскурсию , как вернуть деньги ? Вернут ли мне деньги за оплаченный отель ( гостиницу ) за границей ? Вернуть деньги на самолет ? Вернёт ли мне деньги гостиница ? Могут ли выдать ваучер вместо денег за авиабилет ? \n ? Возврат жд билетов ? Бронировали гостиницу , хотим вернуть деньги ? Как вернуть деньги за невозвратные билеты ? Вернуть деньги за авиабилет ? Вернут ли мне деньги за развлекательные услуги , которые были оплачены иностранной компании ? Бронировали отель , хотим отказаться ? Невозвратные билеты на самолет ? Можно ли вернуть

In [99]:
deeppav_ans = []
for index, row in tqdm(answers.iterrows()):
    text = row['Текст вопросов']
    clean_dp = preprocess_with_deepmipt(text)
    deeppav_ans.append(clean_dp)

In [93]:
answers['preprocess_deepmipt'] = deeppav_ans

In [94]:
answers

,Номер связки,Текст вопросов,Текст ответа,Тематика,preprocess_natasha,preprocess_deepmipt
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы . Могу ли ...
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса ? Сущес...
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа ? Можн...
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ,Могу я отказаться от вакцинации?\nВ каких случ...,Могу я отказаться от вакцинации ? В каких случ...
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ,Безопасна ли вакцинация?\nОпасна ли вакцинация...,Безопасна ли вакцинация ? Опасна ли вакцинация...
5,40,"Что делать, тем кто находится в группе риска?\...","Лицам, входящим в группы риска, следует соблюд...",ГРУППЫ РИСКА,"Что делать, тем кто находится в группе риска?\...","Что делать , тем кто находится в группе риска ..."
6,210,Кто в группе риска по короновирусу?\nКто наход...,Люди всех возрастов рискуют заразиться вирусом...,ГРУППЫ РИСКА,Кто в группе риска по короновирусу?\nКто наход...,Кто в группе риска по короновирусу ? Кто наход...
7,43,Что должны обрабатывать в многоквартирных дома...,"Дезинфекция — это комплекс мероприятий, направ...",ДЕЗИНФЕКЦИЯ ОТКРЫТЫХ ПРОСТРАНСТВ И МНОГОКВАРТИ...,Что должны обрабатывать в многоквартирных дома...,Что должны обрабатывать в многоквартирных дома...
8,82,"Заканчивается виза у иностранного гражданина, ...",В связи с ситуацией по коронавирусу иностранны...,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...","Заканчивается виза у иностранного гражданина, ...","Заканчивается виза у иностранного гражданина ,..."
9,89,Как узнать судьбу сына/родителя/родственника (...,Мы понимаем ваше беспокойство и для получения ...,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",Как узнать судьбу сына/родителя/родственника (...,Как узнать судьбу сына / родителя / родственни...


In [103]:
deeppav_quer = []
for index, row in tqdm(queries.iterrows()):
    text = row['Текст вопроса']
    clean_dp = preprocess_with_deepmipt(text)
    deeppav_quer.append(clean_dp)

In [105]:
queries['preprocess_deepmipt'] = deeppav_quer

In [106]:
queries.tail()

,Текст вопроса,Номер связки\n,Тематика,Unnamed: 3,Unnamed: 4,preprocess_natasha,preprocess_deepmipt
2294,Добрый день.\nГражданин Украины проживает в РФ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN,"Добрый день.\nГражданин проживает в , есть РВ...","Добрый день . Гражданин проживает в , есть . П..."
2295,Добрый вечер. Нахожусь в командировке за грани...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN,Добрый вечер. Нахожусь в командировке за грани...,Добрый вечер . Нахожусь в командировке за гран...
2296,Здравствуйте! \nНам пришёл положительный резул...,1.0,"КАРАНТИН, ИЗОЛЯЦИЯ, САМОИЗОЛЯЦИЯ\n",NaN,NaN,Здравствуйте! \nНам пришёл положительный резул...,Здравствуйте ! Нам пришёл положительный резуль...
2297,"Добрый день, хотелось бы получить консультацию...",45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ\n",NaN,NaN,"Добрый день, хотелось бы получить консультацию...","Добрый день , хотелось бы получить консультаци..."
2298,Добрый день ! \nМоя мама Уланина Валентина Мих...,257.0,"ОКАЗАНИЕ МЕДИЦИНСКОЙ ПОМОЩИ, ГОТОВНОСТЬ СТАЦИО...",NaN,NaN,Добрый день ! \nМоя мама лежит в инфекционном...,Добрый день ! Моя мама лежит в инфекционном от...


### __Задача 2__:    
На предыдущем занятии вы реализовывали функции поиска ближайших ответов на запросы через TF-IDF и BM25. 
Сравните качество нахождения верного ответа для обоих методов в трех случаях:
- с функцией ```preprocess_with_natasha```
- с функцией ```preprocess_with_deepmipt```
- без препроцессинга

Для измерения качества используйте метрику accuracy. Считаем, что ответ верный, если он входит в топ-1.

#### Подготовка к поддсчету

In [72]:
from sklearn.metrics import accuracy_score

In [43]:
# убираем элементы NaN - где нет связок
queries = queries.fillna(0)

In [45]:
for_queries = queries[['Номер связки\n', 'Текст вопроса',
                       'preprocess_natasha',	'preprocess_deepmipt']]

In [46]:
for_queries.rename(columns={'Номер связки\n': 'Номер связки',
                            'Текст вопроса': 'Текст вопросов',
                            'preprocess_natasha': 'preprocess_natasha',
                            'preprocess_deepmipt': 'preprocess_deepmipt'},
                   inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [47]:
for_queries.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt
0,308.0,с уважением Вероника Игоревна Ильич\n\nПосле ...,с уважением \n\nПосле 15 августа 2020 года к н...,с уважением \n \n После 15 августа 2020 года...
1,324.0,"Здравствуйте! Проинформируйте, пожалуйста, нуж...","Здравствуйте! Проинформируйте, пожалуйста, нуж...","Здравствуйте ! Проинформируйте , пожалуйста , ..."
2,57.0,"--\nДобрый день!\n Меня, Сидельникова Андрея...","--\nДобрый день!\n Меня, 30.07.1989г.р., п...","- - \n Добрый день ! Меня , 30 . 07 . 1989г . ..."
3,45.0,Добрый день.\nВ Кемеровской области согласно п...,Добрый день.\nВ согласно постановлению необх...,Добрый день . В согласно постановлению необход...
4,3.0,"Здравствуйте, в моем городе Кострома введено о...","Здравствуйте, в моем городе введено ограничен...","Здравствуйте , в моем городе введено ограничен..."


In [48]:
for_answers = answers[['Номер связки', 'Текст вопросов',
                       'preprocess_natasha',	'preprocess_deepmipt']]

In [49]:
for_answers.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt
0,57,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы . Могу ли ...
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса ? Сущес...
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа ? Можн...
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Могу я отказаться от вакцинации?\nВ каких случ...,Могу я отказаться от вакцинации ? В каких случ...
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,Безопасна ли вакцинация?\nОпасна ли вакцинация...,Безопасна ли вакцинация ? Опасна ли вакцинация...


In [50]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 350

train_df, test = train_test_split(for_queries, test_size=0.3,
                               random_state=RANDOM_STATE)

In [52]:
train_df.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt
1303,132.0,Здравствуйте.Мой муж заболел (1960 г. сердечни...,Здравствуйте.Мой муж заболел (1960 г. сердечни...,Здравствуйте . Мой муж заболел ( 1960 г . серд...
23,286.0,"Здравствуйте, в магазине Хлеб Соль в городе Б...","Здравствуйте, в магазине в городе двое сотр...","Здравствуйте , в магазине в городе двое сотр..."
2055,1.0,"Добрый день. Я, Медведовский Михаил Дмитриевич...","Добрый день. Я, Медведовский , и у моя жена, ,...","Добрый день . Я , , и у моя жена , , 18 . 08 ...."
1800,1.0,---------- Forwarded message ---------\n\nОт: ...,---------- Forwarded message ---------\n\nОт: ...,- - - - - - - - - - Forwarded message - - - - ...
1967,1.0,Добрый день!\n27.08.2020 мной был сдан тест на...,Добрый день!\n27.08.2020 мной был сдан тест на...,Добрый день ! 27 . 08 . 2020 мной был сдан тес...


In [53]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt
2108,1.0,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день ! Сегодня ребенка не допустили к з...
913,308.0,"Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте , по правилам и предписаниям , не..."
2189,79.0,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте ! Хотела узнать стоит ли мне обра...
1438,308.0,"Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро ! Просьба сообщить , обязан ли я с..."
2178,257.0,"Здравствуйте! Мы живём в городе Истра, моей ма...","Здравствуйте! Мы живём в городе , моей маме 67...","Здравствуйте ! Мы живём в городе , моей маме 6..."


In [54]:
train = pd.concat([for_answers, train_df])

In [56]:
!pip install --quiet pymorphy2[fast]

     |████████████████████████████████| 378kB 2.8MB/s 


In [57]:
from pymorphy2 import MorphAnalyzer


morph = MorphAnalyzer()

In [58]:
def preprocessing(data):
    data2 = []
    cnt = 0
    for v in data:
        strng = ''
        line = "".join(l for l in str(v) if l not in (
            '.', ',', '?', '!', ':', ';', '—', '--', '<', '>', '"', '(', ')'))
        for word in line.split():
            word = re.sub(r'\ufeff', '', word)
            wrd = morph.parse(word)[0]
            strng += wrd.normal_form + ' '
        data2.append(strng)
    return data2

In [61]:
lemm_text = preprocessing(tqdm(train['Текст вопросов']))
train['вопросы_препроцессинговые'] = lemm_text

In [62]:
lemm_text2 = preprocessing(tqdm(train['preprocess_natasha']))
train['natasha'] = lemm_text2

In [63]:
lemm_text3 = preprocessing(tqdm(train['preprocess_deepmipt']))
train['deeppavlov'] = lemm_text3

In [64]:
train.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov
0,57.0,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы . Могу ли ...,у ребёнок в школа продлить каникулы мочь ли я ...,у ребёнок в школа продлить каникулы мочь ли я ...,у ребёнок в школа продлить каникулы мочь ли я ...
1,78.0,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса ? Сущес...,где сделать вакцинация от коронавирус существо...,где сделать вакцинация от коронавирус существо...,где сделать вакцинация от коронавирус существо...
2,326.0,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа ? Можн...,сколько стоить сделать вакцина от грипп можно ...,сколько стоить сделать вакцина от грипп можно ...,сколько стоить сделать вакцина от грипп можно ...
3,327.0,Могу я отказаться от вакцинации?\nВ каких случ...,Могу я отказаться от вакцинации?\nВ каких случ...,Могу я отказаться от вакцинации ? В каких случ...,мочь я отказаться от вакцинация в какой случай...,мочь я отказаться от вакцинация в какой случай...,мочь я отказаться от вакцинация в какой случай...
4,328.0,Безопасна ли вакцинация?\nОпасна ли вакцинация...,Безопасна ли вакцинация?\nОпасна ли вакцинация...,Безопасна ли вакцинация ? Опасна ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...


In [65]:
lemm_text4 = preprocessing(tqdm(test['Текст вопросов']))
test['вопросы_препроцессинговые'] = lemm_text4

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
lemm_text5 = preprocessing(tqdm(test['preprocess_natasha']))
test['natasha'] = lemm_text5

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [67]:
lemm_text6 = preprocessing(tqdm(test['preprocess_deepmipt']))
test['deeppavlov'] = lemm_text6

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov
2108,1.0,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день ! Сегодня ребенка не допустили к з...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...
913,308.0,"Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте , по правилам и предписаниям , не...",здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...
2189,79.0,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте ! Хотела узнать стоит ли мне обра...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...
1438,308.0,"Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро ! Просьба сообщить , обязан ли я с...",добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...
2178,257.0,"Здравствуйте! Мы живём в городе Истра, моей ма...","Здравствуйте! Мы живём в городе , моей маме 67...","Здравствуйте ! Мы живём в городе , моей маме 6...",здравствуйте мы жить в город истра мой мама 67...,здравствуйте мы жить в город мой мама 67 год и...,здравствуйте мы жить в город мой мама 67 год и...


#### TF-IDF

##### preprocess_with_natasha

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()

In [70]:
X = vectorizer.fit_transform(train['natasha'].values.astype('U'))

In [71]:
matr_answers = X.toarray()

In [72]:
X

<1652x6503 sparse matrix of type '<class 'numpy.float64'>'
	with 61892 stored elements in Compressed Sparse Row format>

In [73]:
# эту функцию можно было и не использовать, в принципе,
# но я опиралась на 2 домашку, поэтому решила оставить

def preprocess_text(data):
    strng = ''
    line = re.sub(r'[\.\,?!:;—<>\"\'()@#$\*+=]', '', str(data))
    for word in line.split():
        wrd = morph.parse(word)[0]
        strng += wrd.normal_form + ' '
    return strng

In [74]:
def text_to_vec(text, vectorizer, matr):
    txt = preprocess_text(text)
    new_doc = vectorizer.transform([txt]).toarray()
    new_matr = matr.dot(new_doc[0])
    return new_matr

In [75]:
n_conn = train['Номер связки'].tolist()

In [76]:
import operator

In [77]:
def ranging_simple(text):
    
    dict_texts = {}

    cnt = 0
    for x in np.nditer(text):
        dict_texts[n_conn[cnt]] = float(x)
        cnt += 1

    new_dict = sorted(dict_texts.items(),
                      key=operator.itemgetter(1), reverse=True)
    first = []
    for k in new_dict:
        first.append(k[0])

    return first[0]

In [78]:
for_tf_natasha = []
for index, row in tqdm(test.iterrows()):
    fin = text_to_vec(row['natasha'], vectorizer, matr_answers)
    for_tf_natasha.append(ranging_simple(fin))

In [79]:
test['tf_vector_natasha'] = for_tf_natasha

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
from sklearn.metrics import accuracy_score

In [81]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov,tf_vector_natasha
2108,1.0,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день ! Сегодня ребенка не допустили к з...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,12.0
913,308.0,"Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте , по правилам и предписаниям , не...",здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,327.0
2189,79.0,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте ! Хотела узнать стоит ли мне обра...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,132.0
1438,308.0,"Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро ! Просьба сообщить , обязан ли я с...",добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,308.0
2178,257.0,"Здравствуйте! Мы живём в городе Истра, моей ма...","Здравствуйте! Мы живём в городе , моей маме 67...","Здравствуйте ! Мы живём в городе , моей маме 6...",здравствуйте мы жить в город истра мой мама 67...,здравствуйте мы жить в город мой мама 67 год и...,здравствуйте мы жить в город мой мама 67 год и...,32.0


In [82]:
# accuracy_score(y_true, y_pred)

accuracy_score(test['Номер связки'], test['tf_vector_natasha'])

0.22608695652173913

##### preprocess_with_deepmipt

In [83]:
vectorizer2 = TfidfVectorizer()

In [84]:
X2 = vectorizer2.fit_transform(train['deeppavlov'].values.astype('U'))

In [85]:
matr_answers_2 = X2.toarray()

In [86]:
for_tf_deeppavlov = []
for index, row in tqdm(test.iterrows()):
    fin = text_to_vec(row['deeppavlov'], vectorizer2, matr_answers_2)
    for_tf_deeppavlov.append(ranging_simple(fin))

In [87]:
test['tf_vector_deeppavlov'] = for_tf_deeppavlov

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov,tf_vector_natasha,tf_vector_deeppavlov
2108,1.0,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день ! Сегодня ребенка не допустили к з...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,12.0,12.0
913,308.0,"Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте , по правилам и предписаниям , не...",здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,327.0,327.0
2189,79.0,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте ! Хотела узнать стоит ли мне обра...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,132.0,132.0
1438,308.0,"Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро ! Просьба сообщить , обязан ли я с...",добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,308.0,308.0
2178,257.0,"Здравствуйте! Мы живём в городе Истра, моей ма...","Здравствуйте! Мы живём в городе , моей маме 67...","Здравствуйте ! Мы живём в городе , моей маме 6...",здравствуйте мы жить в город истра мой мама 67...,здравствуйте мы жить в город мой мама 67 год и...,здравствуйте мы жить в город мой мама 67 год и...,32.0,32.0


In [90]:
accuracy_score(test['Номер связки'], test['tf_vector_deeppavlov'])

0.2246376811594203

##### без препроцессинга

In [92]:
vectorizer3 = TfidfVectorizer()

In [93]:
X3 = vectorizer3.fit_transform(train['вопросы_препроцессинговые'].values.astype('U'))

In [94]:
matr_answers_3 = X3.toarray()

In [95]:
for_tf_none = []
for index, row in tqdm(test.iterrows()):
    fin = text_to_vec(row['вопросы_препроцессинговые'], vectorizer3, matr_answers_3)
    for_tf_none.append(ranging_simple(fin))

In [96]:
test['tf_vector_no_prep'] = for_tf_none

In [98]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov,tf_vector_natasha,tf_vector_deeppavlov,tf_vector_no_prep
2108,1.0,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день ! Сегодня ребенка не допустили к з...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,12.0,12.0,32.0
913,308.0,"Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте , по правилам и предписаниям , не...",здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,327.0,327.0,327.0
2189,79.0,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте ! Хотела узнать стоит ли мне обра...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,132.0,132.0,132.0
1438,308.0,"Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро ! Просьба сообщить , обязан ли я с...",добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,308.0,308.0,308.0
2178,257.0,"Здравствуйте! Мы живём в городе Истра, моей ма...","Здравствуйте! Мы живём в городе , моей маме 67...","Здравствуйте ! Мы живём в городе , моей маме 6...",здравствуйте мы жить в город истра мой мама 67...,здравствуйте мы жить в город мой мама 67 год и...,здравствуйте мы жить в город мой мама 67 год и...,32.0,32.0,32.0


In [99]:
accuracy_score(test['Номер связки'], test['tf_vector_no_prep'])

0.19855072463768117

#### BM25

Функция bm25, которую я писала к прошлой домашке работала очень долго (даже слишком), поэтому я подумала использовать готовые модули в питоне, которые нашла тут: https://pypi.org/project/rank-bm25/

In [100]:
!pip install rank_bm25

In [101]:
from rank_bm25 import BM25Okapi

##### preprocess_with_natasha

In [102]:
corpus1 = train.natasha.tolist()

In [103]:
tokenized_corpus1 = [doc.split(" ") for doc in corpus1]

In [104]:
bm251 = BM25Okapi(tokenized_corpus1)

In [105]:
for_bm_natasha = []
for index, row in tqdm(test.iterrows()):
    tokenized_query = row.natasha.split(" ")
    doc_scores = bm251.get_scores(tokenized_query)
    for_bm_natasha.append(ranging_simple(doc_scores))

In [106]:
test['bm_natasha'] = for_bm_natasha

In [107]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov,tf_vector_natasha,tf_vector_deeppavlov,tf_vector_no_prep,bm_natasha
2108,1.0,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день ! Сегодня ребенка не допустили к з...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,12.0,12.0,32.0,12.0
913,308.0,"Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте , по правилам и предписаниям , не...",здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,327.0,327.0,327.0,327.0
2189,79.0,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте ! Хотела узнать стоит ли мне обра...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,132.0,132.0,132.0,1.0
1438,308.0,"Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро ! Просьба сообщить , обязан ли я с...",добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,308.0,308.0,308.0,12.0
2178,257.0,"Здравствуйте! Мы живём в городе Истра, моей ма...","Здравствуйте! Мы живём в городе , моей маме 67...","Здравствуйте ! Мы живём в городе , моей маме 6...",здравствуйте мы жить в город истра мой мама 67...,здравствуйте мы жить в город мой мама 67 год и...,здравствуйте мы жить в город мой мама 67 год и...,32.0,32.0,32.0,6.0


In [108]:
accuracy_score(test['Номер связки'], test['bm_natasha'])

0.1391304347826087

##### preprocess_with_deepmipt

In [109]:
corpus2 = train.deeppavlov.tolist()

In [110]:
tokenized_corpus2 = [doc.split(" ") for doc in corpus2]

In [111]:
bm252 = BM25Okapi(tokenized_corpus2)

In [112]:
for_bm_deeppavlov = []
for index, row in tqdm(test.iterrows()):
    tokenized_query = row.deeppavlov.split(" ")
    doc_scores = bm252.get_scores(tokenized_query)
    for_bm_deeppavlov.append(ranging_simple(doc_scores))

In [113]:
test['bm_deeppavlov'] = for_bm_deeppavlov

In [136]:
test.tail()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov,tf_vector_natasha,tf_vector_deeppavlov,tf_vector_no_prep,bm_natasha,bm_deeppavlov,bm_no_prep
1872,1.0,Добрый день!\nМоей супруге находящейся в больн...,Добрый день!\nМоей супруге находящейся в больн...,Добрый день ! Моей супруге находящейся в больн...,добрый день мой супруга находиться в больница ...,добрый день мой супруга находиться в больница ...,добрый день мой супруга находиться в больница ...,74.0,74.0,74.0,74.0,74.0,74.0
262,6.0,Здравствуйте. У меня есть симптомы ОРВИ-насмор...,Здравствуйте. У меня есть симптомы ОРВИ-насмор...,Здравствуйте . У меня есть симптомы ОРВИ - нас...,здравствуйте у я есть симптом орви-насморк гор...,здравствуйте у я есть симптом орви-насморк гор...,здравствуйте у я есть симптом орвать - насморк...,324.0,324.0,286.0,270.0,32.0,324.0
2224,1.0,\nДобрый день! Сегодня моей дочери Волковой Ал...,\nДобрый день! Сегодня моей дочери сообщили ...,Добрый день ! Сегодня моей дочери сообщили из ...,добрый день сегодня мой дочь волков александр ...,добрый день сегодня мой дочь сообщить из школа...,добрый день сегодня мой дочь сообщить из школа...,57.0,57.0,257.0,57.0,57.0,57.0
2245,1.0,Добрый день! Каков порядок контактных людей по...,Добрый день! Каков порядок контактных людей по...,Добрый день ! Каков порядок контактных людей п...,добрый день каков порядок контактный человек п...,добрый день каков порядок контактный человек п...,добрый день каков порядок контактный человек п...,308.0,308.0,308.0,57.0,57.0,57.0
1527,308.0,"Добрый день!\n\nПомогите, пожалуйста, разобрат...","Добрый день!\n\nПомогите, пожалуйста, разобрат...","Добрый день ! Помогите , пожалуйста , разобрат...",добрый день помочь пожалуйста разобраться я ве...,добрый день помочь пожалуйста разобраться я ве...,добрый день помочь пожалуйста разобраться я ве...,308.0,308.0,308.0,79.0,32.0,308.0


In [115]:
accuracy_score(test['Номер связки'], test['bm_deeppavlov'])

0.13623188405797101

##### без препроцессинга

In [116]:
corpus3 = train['вопросы_препроцессинговые'].tolist()

In [117]:
tokenized_corpus3 = [doc.split(" ") for doc in corpus3]

In [118]:
bm253 = BM25Okapi(tokenized_corpus3)

In [119]:
for_bm_no_prep = []
for index, row in tqdm(test.iterrows()):
    tokenized_query = row['вопросы_препроцессинговые'].split(" ")
    doc_scores = bm253.get_scores(tokenized_query)
    for_bm_no_prep.append(ranging_simple(doc_scores))

In [120]:
test['bm_no_prep'] = for_bm_no_prep

In [121]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov,tf_vector_natasha,tf_vector_deeppavlov,tf_vector_no_prep,bm_natasha,bm_deeppavlov,bm_no_prep
2108,1.0,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день ! Сегодня ребенка не допустили к з...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,12.0,12.0,32.0,12.0,12.0,12.0
913,308.0,"Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте , по правилам и предписаниям , не...",здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,327.0,327.0,327.0,327.0,325.0,57.0
2189,79.0,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте ! Хотела узнать стоит ли мне обра...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,132.0,132.0,132.0,1.0,132.0,1.0
1438,308.0,"Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро ! Просьба сообщить , обязан ли я с...",добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,308.0,308.0,308.0,12.0,12.0,12.0
2178,257.0,"Здравствуйте! Мы живём в городе Истра, моей ма...","Здравствуйте! Мы живём в городе , моей маме 67...","Здравствуйте ! Мы живём в городе , моей маме 6...",здравствуйте мы жить в город истра мой мама 67...,здравствуйте мы жить в город мой мама 67 год и...,здравствуйте мы жить в город мой мама 67 год и...,32.0,32.0,32.0,6.0,6.0,32.0


In [122]:
accuracy_score(test['Номер связки'], test['bm_no_prep'])

0.15072463768115943

### __Задача 3__:    
Улучшить правила в natasha. Написать правила, которые ловят даты в следующих примерах и пересчитать статистику из Задачи 2:
- Уехал 8-9 ноября в Сочи
- Уезжаю 5 числа                           
- 20го сентября заболел

Пример можно посмотреть тут: https://github.com/natasha/yargy

In [137]:
!pip install yargy

In [142]:
from yargy import Parser, rule, and_, not_
from yargy.interpretation import fact
from yargy.predicates import gram
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

В написании правила я решила опереться на эту статью: https://habr.com/ru/post/349864/

In [183]:
from yargy import rule, and_, Parser, or_
from yargy.predicates import gte, lte, dictionary

MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'мая': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}

# инициализируем день
DAY = and_(
    gte(1),
    lte(31)
)

# инициализируем месяц
MONTH = and_(
    gte(1),
    lte(12)
)

# инициализируем год
YEAR = and_(
    gte(1),
    lte(2020)
)

MONTH_NAME = dictionary(
    MONTHS
)

DATE = or_(
    rule(YEAR, '-', MONTH, '-', DAY),
    rule(YEAR, '-', MONTH_NAME, '-', DAY),
    rule(DAY, '-', DAY, MONTH_NAME),
    rule(DAY, '-', DAY, 'числа'),
    rule(DAY, 'числа'),
    rule(DAY, 'го', MONTH_NAME),
    rule(DAY, '-го', MONTH),
    rule(DAY, 'числа', YEAR.optional()),
    rule(DAY, 'числа', MONTH_NAME, YEAR.optional())
    )

In [184]:
parser = Parser(DATE)
text = 'Уехал 8-9 ноября в Сочи Уезжаю 5 числа 20го сентября заболел'
t_text = ''
final = []
for match in parser.findall(text):
    t_text = ''
    for el in [_.value for _ in match.tokens]:
        if len(el) < 3:
            t_text += el
        else:
            t_text += ' ' + el
    final.append(t_text)

In [185]:
final

['8-9 ноября', '5 числа', '20го сентября']

In [186]:
def preprocess_with_natasha_data(text: str) -> str:
    text = str(text)

    t_text = ''
    final = []
    for match in parser.findall(text):
        t_text = ''
        for el in [_.value for _ in match.tokens]:
            if len(el) < 3:
                t_text += el
            else:
                t_text += ' ' + el
        final.append(t_text)
    
    clean_text = text
    for el in final:
        clean_text = clean_text.replace(el, '')

    return clean_text

Для одного 

In [187]:
nt = preprocess_with_natasha_data(text)

In [188]:
nt

'Уехал  в Сочи Уезжаю   заболел'

In [189]:
natasha_date = []
for index, row in tqdm(train.iterrows()):
    text = row['preprocess_natasha']
    clean_nat = preprocess_with_natasha_data(text)
    natasha_date.append(clean_nat)

In [190]:
train['preprocess_natasha_date'] = natasha_date

In [191]:
natasha_date_test = []
for index, row in tqdm(test.iterrows()):
    text = row['preprocess_natasha']
    clean_nat = preprocess_with_natasha_data(text)
    natasha_date_test.append(clean_nat)

In [193]:
test['preprocess_natasha_date'] = natasha_date_test

In [194]:
lemm_text_new = preprocessing(tqdm(train['preprocess_natasha_date']))
train['natasha_date'] = lemm_text_new

In [195]:
train.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov,preprocess_natasha_date,natasha_date
0,57.0,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы . Могу ли ...,у ребёнок в школа продлить каникулы мочь ли я ...,у ребёнок в школа продлить каникулы мочь ли я ...,у ребёнок в школа продлить каникулы мочь ли я ...,У ребенка в школе продлили каникулы. Могу ли я...,у ребёнок в школа продлить каникулы мочь ли я ...
1,78.0,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса ? Сущес...,где сделать вакцинация от коронавирус существо...,где сделать вакцинация от коронавирус существо...,где сделать вакцинация от коронавирус существо...,Где сделать вакцинацию от коронавируса?\nСущес...,где сделать вакцинация от коронавирус существо...
2,326.0,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа ? Можн...,сколько стоить сделать вакцина от грипп можно ...,сколько стоить сделать вакцина от грипп можно ...,сколько стоить сделать вакцина от грипп можно ...,Сколько стоит сделать вакцину от гриппа?\nМожн...,сколько стоить сделать вакцина от грипп можно ...
3,327.0,Могу я отказаться от вакцинации?\nВ каких случ...,Могу я отказаться от вакцинации?\nВ каких случ...,Могу я отказаться от вакцинации ? В каких случ...,мочь я отказаться от вакцинация в какой случай...,мочь я отказаться от вакцинация в какой случай...,мочь я отказаться от вакцинация в какой случай...,Могу я отказаться от вакцинации?\nВ каких случ...,мочь я отказаться от вакцинация в какой случай...
4,328.0,Безопасна ли вакцинация?\nОпасна ли вакцинация...,Безопасна ли вакцинация?\nОпасна ли вакцинация...,Безопасна ли вакцинация ? Опасна ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,Безопасна ли вакцинация?\nОпасна ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...


In [196]:
lemm_text_new_2 = preprocessing(tqdm(test['preprocess_natasha_date']))
test['natasha_date'] = lemm_text_new_2

In [197]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov,tf_vector_natasha,tf_vector_deeppavlov,tf_vector_no_prep,bm_natasha,bm_deeppavlov,bm_no_prep,preprocess_natasha_date,natasha_date
2108,1.0,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день!\nСегодня ребенка не допустили к з...,Добрый день ! Сегодня ребенка не допустили к з...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,добрый день сегодня ребёнок не допустить к зан...,12.0,12.0,32.0,12.0,12.0,12.0,Добрый день!\nСегодня ребенка не допустили к з...,добрый день сегодня ребёнок не допустить к зан...
913,308.0,"Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте, по правилам и предписаниям, необ...","Здравствуйте , по правилам и предписаниям , не...",здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,здравствуйте по правило и предписание необходи...,327.0,327.0,327.0,327.0,325.0,57.0,"Здравствуйте, по правилам и предписаниям, необ...",здравствуйте по правило и предписание необходи...
2189,79.0,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте! Хотела узнать стоит ли мне обрат...,Здравствуйте ! Хотела узнать стоит ли мне обра...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,здравствуйте хотеть узнать стоить ли я обратит...,132.0,132.0,132.0,1.0,132.0,1.0,Здравствуйте! Хотела узнать стоит ли мне обрат...,здравствуйте хотеть узнать стоить ли я обратит...
1438,308.0,"Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро!\n\n\n\nПросьба сообщить, обязан л...","Доброе утро ! Просьба сообщить , обязан ли я с...",добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,добрый утро просьба сообщить обязать ли я сдав...,308.0,308.0,308.0,12.0,12.0,12.0,"Доброе утро!\n\n\n\nПросьба сообщить, обязан л...",добрый утро просьба сообщить обязать ли я сдав...
2178,257.0,"Здравствуйте! Мы живём в городе Истра, моей ма...","Здравствуйте! Мы живём в городе , моей маме 67...","Здравствуйте ! Мы живём в городе , моей маме 6...",здравствуйте мы жить в город истра мой мама 67...,здравствуйте мы жить в город мой мама 67 год и...,здравствуйте мы жить в город мой мама 67 год и...,32.0,32.0,32.0,6.0,6.0,32.0,"Здравствуйте! Мы живём в городе , моей маме 67...",здравствуйте мы жить в город мой мама 67 год и...


#### TF-IDF

In [198]:
vectorizer_new = TfidfVectorizer()

In [199]:
X_new = vectorizer_new.fit_transform(train['natasha_date'].values.astype('U'))

In [200]:
matr_answers_new = X_new.toarray()

In [201]:
for_tf_date = []
for index, row in tqdm(test.iterrows()):
    fin = text_to_vec(row['natasha_date'], vectorizer2, matr_answers_2)
    for_tf_date.append(ranging_simple(fin))

In [202]:
test['tf_vector_date_nat'] = for_tf_date

In [203]:
accuracy_score(test['Номер связки'], test['tf_vector_date_nat'])

0.22608695652173913

#### BM25

In [204]:
corpus_new = train.natasha_date.tolist()

In [205]:
tokenized_corpus_new = [doc.split(" ") for doc in corpus_new]

In [206]:
bm25_new = BM25Okapi(tokenized_corpus_new)

In [207]:
for_bm_date = []
for index, row in tqdm(test.iterrows()):
    tokenized_query = row.natasha_date.split(" ")
    doc_scores = bm25_new.get_scores(tokenized_query)
    for_bm_date.append(ranging_simple(doc_scores))

In [208]:
test['bm_date_nat'] = for_bm_date

In [209]:
test.tail()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,natasha,deeppavlov,tf_vector_natasha,tf_vector_deeppavlov,tf_vector_no_prep,bm_natasha,bm_deeppavlov,bm_no_prep,preprocess_natasha_date,natasha_date,tf_vector_date_nat,bm_date_nat
1872,1.0,Добрый день!\nМоей супруге находящейся в больн...,Добрый день!\nМоей супруге находящейся в больн...,Добрый день ! Моей супруге находящейся в больн...,добрый день мой супруга находиться в больница ...,добрый день мой супруга находиться в больница ...,добрый день мой супруга находиться в больница ...,74.0,74.0,74.0,74.0,74.0,74.0,Добрый день!\nМоей супруге находящейся в больн...,добрый день мой супруга находиться в больница ...,74.0,74.0
262,6.0,Здравствуйте. У меня есть симптомы ОРВИ-насмор...,Здравствуйте. У меня есть симптомы ОРВИ-насмор...,Здравствуйте . У меня есть симптомы ОРВИ - нас...,здравствуйте у я есть симптом орви-насморк гор...,здравствуйте у я есть симптом орви-насморк гор...,здравствуйте у я есть симптом орвать - насморк...,324.0,324.0,286.0,270.0,32.0,324.0,Здравствуйте. У меня есть симптомы ОРВИ-насмор...,здравствуйте у я есть симптом орви-насморк гор...,324.0,270.0
2224,1.0,\nДобрый день! Сегодня моей дочери Волковой Ал...,\nДобрый день! Сегодня моей дочери сообщили ...,Добрый день ! Сегодня моей дочери сообщили из ...,добрый день сегодня мой дочь волков александр ...,добрый день сегодня мой дочь сообщить из школа...,добрый день сегодня мой дочь сообщить из школа...,57.0,57.0,257.0,57.0,57.0,57.0,\nДобрый день! Сегодня моей дочери сообщили ...,добрый день сегодня мой дочь сообщить из школа...,57.0,57.0
2245,1.0,Добрый день! Каков порядок контактных людей по...,Добрый день! Каков порядок контактных людей по...,Добрый день ! Каков порядок контактных людей п...,добрый день каков порядок контактный человек п...,добрый день каков порядок контактный человек п...,добрый день каков порядок контактный человек п...,308.0,308.0,308.0,57.0,57.0,57.0,Добрый день! Каков порядок контактных людей по...,добрый день каков порядок контактный человек п...,308.0,57.0
1527,308.0,"Добрый день!\n\nПомогите, пожалуйста, разобрат...","Добрый день!\n\nПомогите, пожалуйста, разобрат...","Добрый день ! Помогите , пожалуйста , разобрат...",добрый день помочь пожалуйста разобраться я ве...,добрый день помочь пожалуйста разобраться я ве...,добрый день помочь пожалуйста разобраться я ве...,308.0,308.0,308.0,79.0,32.0,308.0,"Добрый день!\n\nПомогите, пожалуйста, разобрат...",добрый день помочь пожалуйста разобраться я ве...,308.0,79.0


In [210]:
accuracy_score(test['Номер связки'], test['bm_date_nat'])

0.13768115942028986